# DETECTION OF FAKE NEWS USING MACHINE LEARNING ALGORITHMS

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import re
import string

In [2]:
df_fake = pd.read_csv(r"C:\Users\Anjola\Downloads\Fake.csv")
df_true = pd.read_csv(r"C:\Users\Anjola\Downloads\True.csv")

In [3]:
df_fake.head(5)

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
df_true.head(5)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


Inserting a column called "class" for fake and real news dataset to categories fake and true news. 

In [5]:
df_fake["class"] = 0
df_true["class"] = 1

In [12]:
df_fake.head(5)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [13]:
df_true.head(5)

,title,text,subject,date,class
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


Removing last 10 rows from both the dataset, for manual testing  

In [14]:
df_fake.shape, df_true.shape

((23481, 5), (21417, 5))

In [15]:
df_fake_manual_testing = df_fake.tail(10)
for i in range(23480,23470,-1):
    df_fake.drop([i], axis=0, inplace=True)
df_true_manual_testing = df_true.tail(10)
for i in range(21416,21406,-1):
    df_true.drop([i], axis=0, inplace=True)

In [16]:
df_fake.shape, df_true.shape

((23471, 5), (21407, 5))

Merging the manual testing dataframe in single dataset and save it in a csv file

In [17]:
df_manual_testing = pd.concat([df_fake_manual_testing, df_true_manual_testing] ,axis=0)
df_manual_testing.to_csv("manual_testing.csv")

In [18]:
df_manual_testing.head(10)

,title,text,subject,date,class
23471,Seven Iranians freed in the prisoner swap have...,"21st Century Wire says This week, the historic...",Middle-east,"January 20, 2016",0
23472,#Hashtag Hell & The Fake Left,By Dady Chery and Gilbert MercierAll writers ...,Middle-east,"January 19, 2016",0
23473,Astroturfing: Journalist Reveals Brainwashing ...,Vic Bishop Waking TimesOur reality is carefull...,Middle-east,"January 19, 2016",0
23474,The New American Century: An Era of Fraud,Paul Craig RobertsIn the last years of the 20t...,Middle-east,"January 19, 2016",0
23475,Hillary Clinton: ‘Israel First’ (and no peace ...,Robert Fantina CounterpunchAlthough the United...,Middle-east,"January 18, 2016",0
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",0
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",0
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",0
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",0
23480,10 U.S. Navy Sailors Held by Iranian Military ...,21st Century Wire says As 21WIRE predicted in ...,Middle-east,"January 12, 2016",0


Merging the main fake and true dataframe

In [19]:
df_marge = pd.concat([df_fake, df_true], axis =0 )
df_marge.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [20]:
df_marge.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

#### "title",  "subject" and "date" columns is not required for detecting the fake news, so I am going to drop the columns.

In [21]:
df = df_marge.drop(["title", "subject","date"], axis = 1)

In [22]:
df.isnull().sum()

text     0
class    0
dtype: int64

#### Randomly shuffling the dataframe 

In [23]:
df = df.sample(frac = 1)

In [24]:
df.head()

,text,class
9135,PRESIDENT TRUMP MADE A SURPRISE DETOUR ON HIS ...,0
12792,TEGUCIGALPA (Reuters) - Honduras election tri...,1
13280,"In NBC s attempt to discredit Trump, they ve s...",0
4248,WASHINGTON (Reuters) - President Donald Trump ...,1
12330,,0


In [26]:
df.reset_index(inplace = True)
df.drop(["index"], axis = 1, inplace = True)

In [27]:
df.columns

Index(['text', 'class'], dtype='object')

In [28]:
df.head()

,text,class
0,PRESIDENT TRUMP MADE A SURPRISE DETOUR ON HIS ...,0
1,TEGUCIGALPA (Reuters) - Honduras election tri...,1
2,"In NBC s attempt to discredit Trump, they ve s...",0
3,WASHINGTON (Reuters) - President Donald Trump ...,1
4,,0


#### Creating a function to convert the text in lowercase, remove the extra space, special chr., ulr and links.

In [29]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text

In [25]:
df["text"] = df["text"].apply(wordopt)

#### Defining dependent and independent variable as x and y

In [35]:
x = df["text"]
y = df["class"]

#### Splitting the dataset into training set and testing set. 

In [78]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

#### Convert text to vectors

In [79]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [80]:
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

### 1. Logistic Regression

In [73]:
from sklearn.linear_model import LogisticRegression

In [74]:
LR = LogisticRegression()
LR.fit(xv_train,y_train)

LogisticRegression()

In [75]:
pred_lr=LR.predict(xv_test)

In [76]:
LR.score(xv_test, y_test)

0.9842908604746166

In [43]:
confusion_matrix(y_test, pred_lr)

array([[4615,   62],
       [  36, 4263]], dtype=int64)

from the confusion matrix i got 
0.9864 as the accuracy 
0.9895 as sensitivity/recall then 
0.9869 as the f1 score and 
0.9844 as the precision.

In [77]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.98      0.98     14123
           1       0.98      0.99      0.98     12804

    accuracy                           0.98     26927
   macro avg       0.98      0.98      0.98     26927
weighted avg       0.98      0.98      0.98     26927



### 1b. Logistic Regression Hyperparameter

In [52]:
logModel = LogisticRegression()

In [54]:
param_grid= [
    {'penalty': ['l1','l2','elasticnet','none'],
     'C': np.logspace(-4,4,20),
     'solver': ['lbfgs','newton-cg','liblinear','sag','saga'],
     'max_iter': [100,1000,2500,5000]
}
]

In [56]:
from sklearn.model_selection import GridSearchCV

In [62]:
clf = GridSearchCV(logModel, param_grid = param_grid, cv = 3, verbose = True, n_jobs = -1)

In [65]:
best_clf = clf.fit(xv_train,y_train) 

Fitting 3 folds for each of 1600 candidates, totalling 4800 fits


KeyboardInterrupt: 

### 2. Decision Tree Classification

In [45]:
from sklearn.tree import DecisionTreeClassifier

In [46]:
DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier()

In [47]:
pred_dt = DT.predict(xv_test)

In [48]:
confusion_matrix(y_test, pred_dt)

array([[4665,   12],
       [  15, 4284]], dtype=int64)

from the confusion matrix i got 
0.9957 as the accuracy 
0.9962 as sensitivity/recall then 
0.9958 as the f1 score and 
0.9955 as the precision.

In [49]:
DT.score(xv_test, y_test)

0.9969919786096256

In [50]:
print(classification_report(y_test, pred_dt))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4677
           1       1.00      1.00      1.00      4299

    accuracy                           1.00      8976
   macro avg       1.00      1.00      1.00      8976
weighted avg       1.00      1.00      1.00      8976



### 3. Gradient Boosting Classifier

In [53]:
from sklearn.ensemble import GradientBoostingClassifier

In [54]:
GBC = GradientBoostingClassifier(random_state=0)
GBC.fit(xv_train, y_train)

GradientBoostingClassifier(random_state=0)

In [56]:
pred_gbc = GBC.predict(xv_test)

In [57]:
confusion_matrix(y_test, pred_gbc)

array([[4648,   29],
       [   9, 4290]], dtype=int64)

from the confusion matrix i got 
0.9949 as the accuracy 
0.9974 as sensitivity/recall then 
0.9951 as the f1 score and 
0.9927 as the precision.

In [58]:
GBC.score(xv_test, y_test)

0.9957664884135472

In [59]:
print(classification_report(y_test, pred_gbc))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      4677
           1       0.99      1.00      1.00      4299

    accuracy                           1.00      8976
   macro avg       1.00      1.00      1.00      8976
weighted avg       1.00      1.00      1.00      8976



### 4. Random Forest Classifier

In [60]:
from sklearn.ensemble import RandomForestClassifier

In [61]:
RFC = RandomForestClassifier(random_state=0)
RFC.fit(xv_train, y_train)

RandomForestClassifier(random_state=0)

In [62]:
pred_rfc = RFC.predict(xv_test)

In [63]:
confusion_matrix(y_test, pred_rfc)

array([[4638,   39],
       [  47, 4252]], dtype=int64)

from the confusion matrix i got 
0.9882 as the accuracy 
0.9885 as sensitivity/recall then 
0.9887 as the f1 score and 
0.9889 as the precision.

In [64]:
RFC.score(xv_test, y_test)

0.9904188948306596

In [65]:
print(classification_report(y_test, pred_rfc))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      4677
           1       0.99      0.99      0.99      4299

    accuracy                           0.99      8976
   macro avg       0.99      0.99      0.99      8976
weighted avg       0.99      0.99      0.99      8976



# Model Testing With Manual Entry

### News

In [66]:
def output_lable(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not A Fake News"
    
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt) 
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_GBC = GBC.predict(new_xv_test)
    pred_RFC = RFC.predict(new_xv_test)

    return print("\n\nLR Prediction: {} \nDT Prediction: {} \nGBC Prediction: {} \nRFC Prediction: {}".format(output_lable(pred_LR[0]), 
                                                                                                              output_lable(pred_DT[0]), 
                                                                                                              output_lable(pred_GBC[0]), 
                                                                                                              output_lable(pred_RFC[0])))

In [68]:
news = str(input())
manual_testing(news)

MOSCOW (Reuters) - Vatican Secretary of State Cardinal Pietro Parolin said on Tuesday that there was  positive momentum  behind the idea of Pope Francis visiting Russia, but suggested there was more work to be done if it were to happen.  Parolin, speaking at a joint news conference in Moscow alongside Russian Foreign Minister Sergei Lavrov, did not give any date for such a possible visit. The Eastern and Western branches of Christianity split apart in 1054. The pope, leader of the world s 1.2 billion Catholics, is seeking to improve ties, and last year in Cuba held what was the first ever meeting between a Roman Catholic pope and a Russian Orthodox patriarch.  Parolin said he had also used his talks in the Russian capital to also raise certain difficulties faced by the Catholic Church in Russia. He said that Moscow and the Vatican disagreed about the plight of Christians in certain parts of the world. He did not elaborate. Parolin, who is due later on Tuesday to meet Patriarch Kirill, 